In [148]:
import pandas as pd
import numpy as np

from statsmodels.stats.weightstats import ztest
from datetime import date
from statsmodels.stats.multitest import multipletests

По каждому товару в каждом магазине есть OKR на полгода. Для упрощения задачи представим, что по всем товарам OKR одинаковый — 15 единиц за полгода.

Проверяем гипотезу о том, что если в основном OKR были выполнены по магазину в первом полугодии, то они выполняются и во втором.

Тогда:

H0: нет статистически значимых различий между числом достигнутых OKR в первом и втором полугодиях.
H1: различия есть.

In [149]:
sales = pd.read_csv('/Users/yanaprynik/Downloads/8_модуль.csv',sep=';')
products = pd.read_csv('/Users/yanaprynik/Downloads/products.csv')

sales.head()


,product_id,store_id,date,sales
0,4,6,1997-01-01,4
1,25,6,1997-01-01,3
2,48,6,1997-01-01,3
3,76,6,1997-01-01,4
4,119,6,1997-01-01,3


In [150]:
sales = sales.merge(products[['product_id', 'product_name']], on='product_id', how='inner')
sales['date'] = pd.to_datetime(sales['date'])
sales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 83391 entries, 0 to 83390
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   product_id    83391 non-null  int64         
 1   store_id      83391 non-null  int64         
 2   date          83391 non-null  datetime64[ns]
 3   sales         83391 non-null  int64         
 4   product_name  83391 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 3.8+ MB


сделаем еще один столбец, для различия полугодий

In [151]:
half=[]
for i in sales['date']:
    if i<date(1997,7,1):
        half.append('1')
    else:
        half.append('2')
sales['half_year'] = half

/var/folders/cn/scf3zm9942105gh6p3bt09_00000gn/T/ipykernel_9847/2093670196.py:3: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if i<date(1997,7,1):


Сгруппируем таблицу по необходимым показателям

In [152]:
new_df = sales.groupby(['store_id', 'product_name', 'half_year'])['sales'].sum().reset_index()
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31607 entries, 0 to 31606
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   store_id      31607 non-null  int64 
 1   product_name  31607 non-null  object
 2   half_year     31607 non-null  object
 3   sales         31607 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 987.8+ KB


Сделаем столбец с выполнением плана (0 - не выполнено, 1 - выполнено)

In [153]:
okr = []

for i in new_df['sales']:
    if i >= 15:
        okr.append(1)
    else:
        okr.append(0)
        
new_df['okr'] = okr
new_df.head()

,store_id,product_name,half_year,sales,okr
0,2,ADJ Rosy Sunglasses,1,3,0
1,2,Akron City Map,1,1,0
2,2,Akron Eyeglass Screwdriver,1,1,0
3,2,Akron Eyeglass Screwdriver,2,2,0
4,2,American Beef Bologna,1,4,0


посчитаем p-value для каждого магазина и кол-во выполненого плана

In [154]:
test = {
    'store_id': [],
    'first_half': [],
    'second_half': [],
    'p_value': []
    }

for store, group in new_df.groupby('store_id'):
    first_half = group[group['half_year']=='1']['okr'].values
    second_half = group[group['half_year']=='2']['okr'].values
    
    statistic, p_value = ztest(first_half, second_half)

    test["store_id"].append(store)
    test["first_half"].append(np.sum(first_half))
    test["second_half"].append(np.sum(second_half))
    test["p_value"].append(p_value)
    
df_test = pd.DataFrame(test)
df_test

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/stats/weightstats.py:748: RuntimeWarning: invalid value encountered in scalar divide
  zstat = (value1 - value2 - diff) / std_diff


,store_id,first_half,second_half,p_value
0,2,0,0,NaN
1,3,155,173,0.427237
2,6,85,142,0.000623
3,7,153,225,0.000313
4,11,205,172,0.090413
5,13,626,609,0.503724
6,14,0,0,NaN
7,15,159,188,0.133241
8,16,113,162,0.004140
9,17,387,517,0.000001


посмотрим сколько гипотез Н0 опровергнуть

In [155]:
df_test[df_test.p_value <= 0.05].shape

(5, 4)

Далее скорректируем методом Бонферрони наши гипотезы 

In [156]:
multipletests(df_test.p_value, alpha=0.05, method= 'bonferroni')[0].sum()

3

с помощью метода Холма-Бонферрони еще раз посмотрим 

In [157]:
multipletests(df_test.p_value, alpha=0.05, method= 'holm')[0].sum()

4

Таким образом видим, что примерно 3-4 теории H0 можно опровергнуть, что довольно значимый показатель для 13 гипотез. Из таблицы видим, что 3 магазина почти ничего не продали, скорее всего только открываются или уже закрыты. Поэтому множно выдвинуть гипотезу, что почит половина магазинов имеет статистические различия и не сможет выполнить план в каком-то из полугодий. 